<a href="https://colab.research.google.com/github/AhmedWael2000/TF-IDF/blob/main/TF_IDF_from_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
nltk.download("wordnet")
from nltk.stem import PorterStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
def get_text_webpage(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    text = soup.get_text()
    return text

In [ ]:
url = "https://www.shakespeare-online.com/plays/"
og_text = get_text_webpage(url)
og_text

'\n\nShakespeare\'s Plays\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\ndirectory\n\n\n\n\n\n\nhome\ncontact\n\n\n\n\n\n\n\n\n\n\nwelcome\nplays\nsonnets\nanalysis\nquotations\nsources\nbiography\ntheatres\nkey dates\nplots\nfaq\nbooks\nglossary\nscholars\nquiz\n\n\n\n\nsearch\n\n\n\n\n\n\n\n\n\n\n\xa0\n\n\n\n\n\n\nShakespeare\'s Plays\r\nBefore the publication of the First Folio in 1623, nineteen of the thirty-seven plays in Shakespeare\'s canon had appeared in quarto format. With the exception of Othello (1622), all of the quartos were published prior to the date of Shakespeare\'s retirement from the theatre in about 1611. It is unlikely that Shakespeare was involved directly with the printing of any of his plays, although it should be noted that two of his poems, Venus and Adonis and The Rape of Lucrece were almost certainly printed under his direct supervision. \r\n\r\nHere you will find the complete text of Shakespeare\'s plays, based primarily on the First Folio, and a

In [ ]:
punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ''' + string.punctuation

# English stop words with nltk
stop_words = stopwords.words('english')

lemmatizer = WordNetLemmatizer()

stemmer = PorterStemmer()

In [ ]:
def preprocess(text):
    #remove punctuations / cleaning
    translator = str.maketrans('', '', punctuations)
    text = text.translate(translator)
    #remove English stopping words & Normalization
    text = ' '.join(word.lower() for word in text.split() if word not in stop_words)
    # tokenization
    words = nltk.word_tokenize(text)
    # lemmatization
    lemmas = [lemmatizer.lemmatize(word) for word in words]
    #stemming
    stems = [stemmer.stem(lemma) for lemma in lemmas]
    return stems


In [ ]:
processed_list = preprocess(og_text)
processed_list

In [ ]:
def get_unique_words(
    words:list
    ) -> list:
    return [*set(words)]

In [ ]:
unique_list = get_unique_words(processed_list)
unique_list

In [ ]:
len(processed_list), len(unique_list)

(1629, 750)

In [ ]:
# prompt: from sklearn import tfidf
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectorizer.fit(og_text)
X = vectorizer.transform(unique_list)
print(X.shape)
df = pd.DataFrame(X)

#Imports and doc

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import math

# this is a very toy example, do not try this at home unless you want to understand the usage differences
# docs=[ "Hany love going to school",
#                  "The school  is far from Sara home",
#                  "Hany likes apple more than banana",
#                  "Sara likes apple too"]
docs=[ "the cats are in the house",
                 "the dogs are in the house and outside",
                 "the cats and dogs are friends"]

# Get Unique words

In [ ]:
uniq_words =[]
for i in docs:
  uniq_words.extend(i.split())

In [ ]:
uniq_words = list(set(uniq_words))
len(uniq_words)


9

#Utils

In [ ]:
def table(data,index):
  return pd.DataFrame(data, index=index)


In [ ]:
def sentExist(word,sent):
  count=0
  for i in sent.split():
    if i == word:
      count+=1

  return count

In [ ]:
def docsExist(word,docs):
  counter = 0
  for i in docs:
    if word in i.split():
      counter +=1
  return counter



#TF (Term Frequency)

In [ ]:
def tf(uniq_words,docs):
  tf_mat = np.zeros((len(uniq_words),len(docs)))
  for i in range(len(uniq_words)):
    for j in range(len(docs)):
      tf_mat[i,j] = sentExist(uniq_words[i],docs[j])/len(docs[j].split())

  return tf_mat


In [ ]:
def tf_count(uniq_words,docs):
  tf_mat = np.zeros((len(uniq_words),len(docs)))
  for i in range(len(uniq_words)):
    for j in range(len(docs)):
      tf_mat[i,j] = (sentExist(uniq_words[i],docs[j]))

  return tf_mat.astype(int)

#IDF

In [ ]:
def idf_single( unique_words,all_docs):

    n_docs = len(all_docs)
    len_words = len(unique_words)
    idf = np.zeros(len_words)
    for i in range(len(unique_words)):
        frq = docsExist(uniq_words[i],docs)
        idf[i] = math.log(float(n_docs + 1) / float(frq + 1)) + 1
    return idf.reshape(-1,1)

In [ ]:
def idf(uniq_words,docs):
  n = len(docs)
  tf_mat = np.zeros((len(uniq_words),len(docs)))
  for i in range(len(uniq_words)):
    for j in range(len(docs)):
      if uniq_words[i] in docs[j]:
        frq = docsExist(uniq_words[i],docs)
        tf_mat[i,j] = np.log10(n / frq)
        # tf_mat[i,j] = np.log10(n/docsExist(uniq_words[i],docs))
  return tf_mat

# Applying TF - IDF

In [ ]:
# tf
pd.DataFrame(tf_count(uniq_words,docs),index = uniq_words).T

,the,in,house,cats,dogs,are,and,friends,outside
0,2,1,1,1,0,1,0,0,0
1,2,1,1,0,1,1,1,0,1
2,1,0,0,1,1,1,1,1,0


In [ ]:
# idf
pd.DataFrame(idf_single(uniq_words,docs),index = uniq_words).T

,the,in,house,cats,dogs,are,and,friends,outside
0,1.0,1.287682,1.287682,1.287682,1.287682,1.0,1.287682,1.693147,1.693147


In [ ]:
# tf - idf with normalization
tf_= tf_count(uniq_words,docs)
idf_=idf_single(uniq_words,docs)
tf_idf = idf_*tf_
norm  =np.sqrt((tf_idf*tf_idf).sum(axis=0))
pd.DataFrame(tf_idf/norm,index = uniq_words).T

,the,in,house,cats,dogs,are,and,friends,outside
0,0.633267,0.407724,0.407724,0.407724,0.000000,0.316634,0.000000,0.000000,0.000000
1,0.525239,0.338171,0.338171,0.000000,0.338171,0.262620,0.338171,0.000000,0.444654
2,0.318770,0.000000,0.000000,0.410475,0.410475,0.318770,0.410475,0.539725,0.000000


#Using Built in TFIDF Vectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
# docs=[ "Hany love going to school",
#                  "The school  is far from Sara home",
#                  "Hany likes apple more than banana",
#                  "Sara likes apple too"]

# settings that you use for count vectorizer will go here
tfidf_vectorizer=TfidfVectorizer(use_idf=True)

# just send in all your docs here
tfidf_vectorizer_vectors=tfidf_vectorizer.fit_transform(docs)

# get the first vector out (for the first document)
first_vector_tfidfvectorizer=tfidf_vectorizer_vectors[0]

# place tf-idf values in a pandas data frame
df = pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), index=tfidf_vectorizer.get_feature_names_out(), columns=["tfidf"])

In [ ]:
tfidf_vectorizer=TfidfVectorizer(use_idf=True)

# just send in all your docs here
fitted_vectorizer=tfidf_vectorizer.fit(docs)
tfidf_vectorizer_vectors=fitted_vectorizer.transform(docs)

In [ ]:
table(tfidf_vectorizer_vectors.T.toarray(),tfidf_vectorizer.get_feature_names_out()).T

,and,are,cats,dogs,friends,house,in,outside,the
0,0.000000,0.316634,0.407724,0.000000,0.000000,0.407724,0.407724,0.000000,0.633267
1,0.338171,0.262620,0.000000,0.338171,0.000000,0.338171,0.338171,0.444654,0.525239
2,0.410475,0.318770,0.410475,0.410475,0.539725,0.000000,0.000000,0.000000,0.318770
